In [24]:
import os, pandas, numpy as np
from amplpy import AMPL

In [ ]:
# Initialize AMPL object
ampl = AMPL()

# Directory where the instance files are located
instance_dir = "instances"

# Ask user for the instance number to solve
instance = int(input("Enter the instance number to solve (1-21): "))
instance_file = os.path.join(instance_dir, f"inst{instance:02d}.dat") #_transformed

# Read the first line of the instance file to get the value of n
with open(instance_file, 'r') as file:
    _ = file.readline()
    first_line = file.readline().strip()
    n = int(first_line)
    D = []
    _ = file.readline()
    _ = file.readline()


    for i in range(n+1):
        d_split = file.readline().split()
        d_row = []
        for j in d_split:
            d_row.append(int(j))
        D.append(d_row)
    Dnp = np.array(D)
        

# # Check if the matrix is symmetric
# is_symmetric = np.array_equal(D, D.T)
# print(f"The matrix D is symmetric: {is_symmetric}")

# # Load the AMPL model
# if is_symmetric:
#     ampl.read("model1_sb.mod")
# else:
#     ampl.read("model1.mod")
ampl.read("mod_new.mod")

#Starting from D compute Upper and Lower bound for the problem:
round_trips=[]
for i in range(n):
    round_trips.append(Dnp[i][n]+Dnp[n][i])
LB = max(round_trips)
#LLB = min(round_trips)

UB = 0
for i in range(n):
    UB+=max(Dnp[i,:])
constraint_str1 = f"subject to UB: maxDist <= {UB};"
constraint_str2 = f"subject to LB: maxDist >= {LB};"

ampl.eval(constraint_str1)
ampl.eval(constraint_str2)

# for constraint in ampl.get_constraints():
#     print(constraint[1])

# Load the data for the current instance
instance_file = os.path.join(instance_dir, f"inst{instance:02d}_transformed.dat")
ampl.readData(instance_file)

1455 226
subject to Capacity{i in 1 .. m} : sum{j in 1 .. n} ( if B[j] == i then 1)*s[j
  ] <= l[i];
subject to EndAtDepot: sum{i in 1 .. n} A[i,n + 1] == sum{k in 1 .. m} k;
subject to FlowConservation{i in 1 .. n + 1} : sum{j in 1 .. n + 1} A[i,j]
   == sum{j in 1 .. n + 1} A[j,i];
subject to LB: maxDist >= 226;
subject to LowerBound_u{j in 1 .. n} : u[j] >= s[j];
subject to MaxDistanceConstraint{k in 1 .. m} : sum{i in 1 .. n + 1, j in 1
   .. n + 1} ( if A[i,j] == k then 1)*D[i,j] <= maxDist;
subject to NoBounceBack{i in 1 .. n + 1, j in 1 .. n + 1} : A[i,j]*A[j,i]
   == 0;
subject to NoSelfLoops{i in 1 .. n + 1} : A[i,i] == 0;
subject to NoSubtourTry{i in 1 .. n} : sum{j in 1 .. n + 1} A[i,j] == B[i];
subject to StartAtDepot: sum{i in 1 .. n} A[n + 1,i] == sum{k in 1 .. m} k;
subject to UB: maxDist <= 1455;
subject to VisitOnce{j in 1 .. n} : sum{i in 1 .. n + 1} A[i,j] == B[j];


In [ ]:
# Solve the model
#ampl.solve(solver="cbc", cbc_options="seconds=2 timing=2")
ampl.solve(solver="highs", highs_options="timelim=300 timing=2")

# Get the solution for variables (e.g., x, u, maxDist)
A = ampl.getVariable("A").getValues()
B = ampl.getVariable("B").getValues()

u = ampl.getVariable("u").getValues()
maxDist = ampl.getVariable("maxDist").value()

# Print the results (or process them as needed)
print(f"\nResults for instance inst{instance:02d}.dat:")
print(f"A: {A}")
print(f"B: {B}")
print(f"u: {u}")
print(f"maxDist: {maxDist}")
print("-" * 40)